In [1]:
import os
import sys

# Init project path
PROJECT_DIR = os.getcwd() + "/../"
sys.path.insert(0, PROJECT_DIR)
%load_ext autoreload
%autoreload 2

In [14]:
from models import LightGBM

if len(sys.argv) == 2:
    config_file_name = sys.argv[1]
else:
    config_file_name = "lightgbm_0"

print("Config file Name: ", config_file_name)

# Parse args
config: dict = json.load(open("./../configs/{}.json".format(config_file_name)))
    
models: dict = {"lightgbm": LightGBM}
model = models[config["model"]["name"]]()

Config file Name:  lightgbm_0
